<a href="https://colab.research.google.com/github/katti97/learningdatascience/blob/master/KMeans_clustering_of_Big_data_movie_reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
#Importing all the necessary libraries to unzip files

import gzip
import pandas as pd
import os
import json

In [19]:
#importing TFIDF Vectorizer, Count vectorizer, Nearest Neighbours for text analysis of
#overview of the movies

from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.neighbors import NearestNeighbors

In [20]:
import pickle
# Mounting Google drive to this file
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:
# Install JDK
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [22]:
!wget -q https://downloads.apache.org/spark/spark-3.0.1/spark-3.0.1-bin-hadoop2.7.tgz

In [23]:
!tar -xvf spark-3.0.1-bin-hadoop2.7.tgz

spark-3.0.1-bin-hadoop2.7/
spark-3.0.1-bin-hadoop2.7/RELEASE
spark-3.0.1-bin-hadoop2.7/examples/
spark-3.0.1-bin-hadoop2.7/examples/src/
spark-3.0.1-bin-hadoop2.7/examples/src/main/
spark-3.0.1-bin-hadoop2.7/examples/src/main/scala/
spark-3.0.1-bin-hadoop2.7/examples/src/main/scala/org/
spark-3.0.1-bin-hadoop2.7/examples/src/main/scala/org/apache/
spark-3.0.1-bin-hadoop2.7/examples/src/main/scala/org/apache/spark/
spark-3.0.1-bin-hadoop2.7/examples/src/main/scala/org/apache/spark/examples/
spark-3.0.1-bin-hadoop2.7/examples/src/main/scala/org/apache/spark/examples/ml/
spark-3.0.1-bin-hadoop2.7/examples/src/main/scala/org/apache/spark/examples/ml/FPGrowthExample.scala
spark-3.0.1-bin-hadoop2.7/examples/src/main/scala/org/apache/spark/examples/ml/GBTExample.scala
spark-3.0.1-bin-hadoop2.7/examples/src/main/scala/org/apache/spark/examples/ml/ALSExample.scala
spark-3.0.1-bin-hadoop2.7/examples/src/main/scala/org/apache/spark/examples/ml/KMeansExample.scala
spark-3.0.1-bin-hadoop2.7/example

In [24]:
!pip install -q findspark

In [25]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.1-bin-hadoop2.7"

In [26]:
# Create local Spark session


import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [28]:

import matplotlib.pyplot as plt

# %matplotlib inline
plt.style.use("ggplot")

In [29]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [30]:
#Movies metadata downloaded from Kaggle stored in the drive
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/movies_metadata.csv')

print(len(data))

# First five rows of dataset
data.head()

45466


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,FALSE,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.9469,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",10/30/1995,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,FALSE,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.0155,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",12/15/1995,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,FALSE,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",12/22/1995,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,FALSE,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.85949,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",12/22/1995,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,FALSE,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.38752,/e64sOI48hQXyru7naBFyssKFxVd.jpg,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",2/10/1995,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [31]:
#count of rows in the columns
data.count()

adult                    45466
belongs_to_collection     4494
budget                   45466
genres                   45466
homepage                  7782
id                       45466
imdb_id                  45449
original_language        45455
original_title           45466
overview                 44512
popularity               45461
poster_path              45080
production_companies     45463
production_countries     45463
release_date             45379
revenue                  45460
runtime                  45203
spoken_languages         45460
status                   45379
tagline                  20412
title                    45460
video                    45460
vote_average             45460
vote_count               45460
dtype: int64

In [32]:
#existing columns in the dataset
data.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [33]:
#Creating a subset of dataset with movie title and it's overview

newdata = data.dropna(axis=0, subset=['overview'])
newdata=newdata[['original_title','overview']]
newdata

,original_title,overview
0,Toy Story,"Led by Woody, Andy's toys live happily in his ..."
1,Jumanji,When siblings Judy and Peter discover an encha...
2,Grumpier Old Men,A family wedding reignites the ancient feud be...
3,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom..."
4,Father of the Bride Part II,Just when George Banks has recovered from his ...
...,...,...
45461,رگ خواب,Rising and falling between a man and woman.
45462,Siglo ng Pagluluwal,An artist struggles to finish his work while a...
45463,Betrayal,"When one of her hits goes wrong, a professiona..."
45464,Satana likuyushchiy,"In a small town live two brothers, one a minis..."


In [34]:
#Using sklearn's TFIDF's vectorizer to build a vectorizer by removing stop words of english

vectorizer = TfidfVectorizer(stop_words='english')
vec = vectorizer.fit_transform(newdata['overview'])
vec

<44512x75827 sparse matrix of type '<class 'numpy.float64'>'
	with 1210882 stored elements in Compressed Sparse Row format>

In [35]:
#importing necessary libraraies from pyspark

from pyspark import SparkContext
from pyspark import SparkConf
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.functions import *
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark.ml import Pipeline

from pyspark import SparkConf, SparkContext
from pyspark.mllib.feature import HashingTF
from pyspark.mllib.feature import IDF

!pip install -q findspark

import findspark
findspark.init()

In [36]:
#creating a spark data frame from the dataset that existed and displating the top 20 rows of it

spark = SparkSession.builder.appName("Python Spark SQL basic example").config("spark.some.config.option", "some-value").getOrCreate()

sdf = spark.createDataFrame(newdata.astype(str))
sdf.show()
sdf.cache()

+--------------------+--------------------+
|      original_title|            overview|
+--------------------+--------------------+
|           Toy Story|Led by Woody, And...|
|             Jumanji|When siblings Jud...|
|    Grumpier Old Men|A family wedding ...|
|   Waiting to Exhale|Cheated on, mistr...|
|Father of the Bri...|Just when George ...|
|                Heat|Obsessive master ...|
|             Sabrina|An ugly duckling ...|
|        Tom and Huck|A mischievous you...|
|        Sudden Death|International act...|
|           GoldenEye|James Bond must u...|
|The American Pres...|Widowed U.S. pres...|
|Dracula: Dead and...|When a lawyer sho...|
|               Balto|An outcast half-w...|
|               Nixon|An all-star cast ...|
|    Cutthroat Island|Morgan Adams and ...|
|              Casino|The life of the g...|
|Sense and Sensibi...|Rich Mr. Dashwood...|
|          Four Rooms|It's Ted the Bell...|
|Ace Ventura: When...|Summoned from an ...|
|         Money Train|A vengeful

DataFrame[original_title: string, overview: string]

In [37]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, CountVectorizer, StopWordsRemover
from pyspark.ml.evaluation import ClusteringEvaluator
from pandas.core.reshape.concat import concat

#tokeinizing words from the overview column
tok = Tokenizer(inputCol="overview", outputCol="tokens")
#stop words are removed from overview
stremover = StopWordsRemover(inputCol="tokens", outputCol="stopWordsRemovedTokens")
#hashing is performed
hashingTF = HashingTF(inputCol="stopWordsRemovedTokens", outputCol="rawFeatures", numFeatures=2000)
idf = IDF(inputCol="rawFeatures", outputCol="features", minDocFreq=5)

In [38]:
#12 is found to be the optimum no. of clusters using ClusteringEvaluator
kmeans = KMeans(k=12)
pipeline = Pipeline(stages=[tok, remover, hashingTF, idf, kmeans])

In [39]:
model = pipeline.fit(sdf)

In [40]:
results = model.transform(sdf)

In [41]:
results.show()

+--------------------+--------------------+--------------------+----------------------+--------------------+--------------------+----------+
|      original_title|            overview|              tokens|stopWordsRemovedTokens|         rawFeatures|            features|prediction|
+--------------------+--------------------+--------------------+----------------------+--------------------+--------------------+----------+
|           Toy Story|Led by Woody, And...|[led, by, woody,,...|  [led, woody,, and...|(2000,[87,189,295...|(2000,[87,189,295...|         1|
|             Jumanji|When siblings Jud...|[when, siblings, ...|  [siblings, judy, ...|(2000,[0,72,170,2...|(2000,[0,72,170,2...|         1|
|    Grumpier Old Men|A family wedding ...|[a, family, weddi...|  [family, wedding,...|(2000,[66,104,129...|(2000,[66,104,129...|         1|
|   Waiting to Exhale|Cheated on, mistr...|[cheated, on,, mi...|  [cheated, on,, mi...|(2000,[86,111,417...|(2000,[86,111,417...|         0|
|Father of th

In [42]:
#the movies and their corresponding clusters are as shown below

sdf1 = results.select("original_title","prediction")
sdf2 = sdf1.toPandas()
sdf2 = sdf2.groupby(['prediction'])['original_title'].apply(lambda x: ','.join(x)).reset_index()
sdf2 = sdf2.rename(columns={"prediction": "Cluster Number", "original_title": "Movie name"})
sdf2

,Cluster Number,Movie name
0,0,"Waiting to Exhale,Sabrina,GoldenEye,The Americ..."
1,1,"Toy Story,Jumanji,Grumpier Old Men,Father of t..."
2,2,Black Friday
3,3,"Across the Sea of Time,Restoration,Guardian An..."
4,4,"Wide Eyed and Legless,Of Love and Shadows,A Mo..."
5,5,"StarStruck,Dance Party, USA,Que Horas Ela Volt..."
6,6,Dancing Outlaw II: Jesco Goes to Hollywood
7,7,"De Hel Van Tanger,De Indringer,Windkracht 10: ..."
8,8,"重慶森林,Heavyweights,Como agua para chocolate,Man..."
9,9,The Case of the Howling Dog


In [43]:
sdf2.to_csv(r'/content/drive/My Drive/recom.csv')
sdf2 = pd.read_csv('/content/drive/My Drive/recom.csv')

In [44]:
#function to search for a given movie from the clusters and return top 20 movie suggestions

def search(a):
  l = [(a, )]
  gh = spark.createDataFrame(l, ['overview'])
  pred = model.transform(gh)
  output = pred.collect()[0]['prediction']
  fin = sdf2.loc[sdf2['Cluster Number'] == output]
  fin1 = fin['Movie name'].values[0]
  fin1 = fin1.split(',')
  return fin1

In [45]:
def unique(a):
  uni = []
  for i in a:
    if i not in uni:
      uni.append(i)
  return uni

In [46]:
#Displaying top 20 results for the cluster that the movie belongs to

a = input("Enter movie names to find similar movies: ")
search(a)
unique(search(a))[:20]

Enter movie names to find similar movies: Nixon


['Waiting to Exhale',
 'Sabrina',
 'GoldenEye',
 'The American President',
 'Dracula: Dead and Loving It',
 'Balto',
 'Nixon',
 'Casino',
 'Sense and Sensibility',
 'Four Rooms',
 'Money Train',
 'Copycat',
 'Leaving Las Vegas',
 'Othello',
 'La Cité des Enfants Perdus',
 '摇啊摇，摇到外婆桥',
 'It Takes Two',
 'Cry',
 ' the Beloved Country',
 'Richard III']